In [1]:
# Importing packages
import numpy as np
from sympy import solve, symbols, Eq
import random 
from tqdm import tqdm

# Importing openfermion
from openfermion.ops import FermionOperator
from openfermion.transforms import jordan_wigner, bravyi_kitaev


In [36]:
# Defining Pauli matrices and the identity matrix
I = np.array([[1, 0], [0, 1]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

# Defining the Hamiltonian
H = np.kron(Z, Z) + np.kron(X, X)

# Defining the measurement choices for Alice and Bob
A_0 = X
A_1 = Z
B_0 = 1/np.sqrt(2)*(X+Z)
B_1 = 1/np.sqrt(2)*(X-Z)

# Defining expansion coefficients C_ij
coeffs = [C_00, C_01, C_10, C_11] = symbols(('C_00', 'C_01', 'C_10', 'C_11'))

# Defining the Bell operator
Bell_operator = coeffs[0]*np.kron(A_0, B_0) + coeffs[1]*np.kron(A_0, B_1) + coeffs[2]*np.kron(A_1, B_0) + coeffs[3]*np.kron(A_1, B_1)

# Creating the sytsem of equations
Ops = [X, Z]
eqs = []
for i in range(len(Ops)):
    for j in range(len(Ops)):
        # Defining the projector operator
        Projector = np.kron(Ops[i],Ops[j])

        # calculating system of equations
        eqs.append( 
            Eq( 
                np.trace( np.matmul(Bell_operator, Projector) ), 
                np.trace( np.matmul(H, Projector) )
            ) 
        )

# Solving the system of equations
ans = solve(eqs, coeffs)

# Substituting the results in the Bell operator
Bell_operator = float(ans[C_00])*np.kron(A_0, B_0) + float(ans[C_01])*np.kron(A_0, B_1) + float(ans[C_10])*np.kron(A_1, B_0) + float(ans[C_11])*np.kron(A_1, B_1)

# Calculating the eigenvalues and eigenstates
eig_vals, eig_vecs = np.linalg.eigh(H)

# extracting the ground state energy and the respective eigenstate
E_G = eig_vals[0]
psi_G = eig_vecs[:,0]
psi_G_dagger = np.conjugate(np.transpose(psi_G))

# Adding the ground state energy to the Bell operator 
# Bell_operator += -E_G * np.kron(I,I)

# Calculating the quantum value
beta_Q = np.matmul(psi_G_dagger, np.matmul(Bell_operator, psi_G))

# Calculating the classical bound
m = [-1, 1]
I_Bell = []
for A_0 in m:
    for A_1 in m:
        for B_0 in m:
            for B_1 in m:
                I_Bell.append( 
                    float(ans[C_00])*A_0*B_0 + float(ans[C_10])*A_1*B_0 + float(ans[C_01])*A_0*B_1 + float(ans[C_11])*A_1*B_1
                 )

beta_C = -np.min(I_Bell)

print(r'beta_Q = %.3f and beta_C = %.3f' %(beta_Q, beta_C))


beta_Q = -2.000 and beta_C = 1.414


In [519]:
# Time to obtain the systems of equations and trying to solve it

# Initializing the system of equations
eqs = []

# Looping over the possible Pauli chains
# for j in range()

np.trace(np.matmul(B, get_Pauli_chain([X,Z,X,I]))).simplify()

16.0*c_{1310}

In [520]:
# Creating a list of all possible indices
N_meas = len(M1)
indices = [[i, j, k, l] for i in range(N_meas)]